In [1]:
import pandas as pd
import os 
import dask.dataframe as dd
import numpy as np
from dask.diagnostics import ProgressBar
import gc
import glob
import matplotlib.pyplot as plt
import math
import seaborn as sns
import sys
import re
import rdata
import statistics
import scipy.stats
import collections

from decimal import Decimal, ROUND_HALF_UP

from statistics import mean, median,variance,stdev

from sklearn.metrics import mean_squared_error

In [3]:
df_All_Path = r"df_all_fit_filterd.csv"

usecols = ['FIT_Test_Value', 'ALTC', 'EnvCount', 'Gene','Temperature', 'Radiation', 'FIT_Reference_Value', 'Date']


df = dd.read_csv(df_All_Path,assume_missing=True, usecols=usecols,dtype={'FIT_Test_Value': float,'ALTC': str,'EnvCount': str,
'Gene': str, "Temperature":float , "Radiation":float ,"FIT_Reference_Value": float}).compute()

df['Date(NoHour)'] = pd.to_datetime(df['Date']).dt.date

In [5]:
Corr_df = df.groupby(['ALTC', 'Gene' , "Date(NoHour)"]).apply(lambda x: x['FIT_Test_Value'].corr(x['FIT_Reference_Value'])).reset_index(name='Corr')

def calculate_rmse(group):
    actual = group['FIT_Reference_Value']
    predicted = group['FIT_Test_Value']
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    return rmse

RMSE_df = df.groupby(['ALTC', 'Gene' , "Date(NoHour)"]).apply(calculate_rmse).reset_index(name='RMSE')

df_all = pd.merge(Corr_df,RMSE_df,how="inner",on = ['ALTC', 'Gene' , "Date(NoHour)"])

In [6]:
df_all['Corr_Success'] = df_all['Corr'].apply(lambda x: 1 if x > 0.8 else 0)

df_all['RMSE_Success'] = df_all['RMSE'].apply(lambda x: 1 if x < 0.4 else 0)

df_all['Pridiciton_Success'] = ((df_all['Corr'] > 0.8) & (df_all['RMSE'] < 0.4)).astype(int)

In [7]:
df_all.to_csv(r"df_Pridiciton_Success_Day.csv", index = False)